In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import sys
import scipy
import numpy as np
import matplotlib
import pandas as pd 
import sklearn
import tensorflow as tf

/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/maguileraf23/.conda/envs/deeplift/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/maguileraf23/.conda/envs/

In [2]:
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier #discriminative and supervised
from time import time
from sklearn.feature_selection import SelectFromModel

# Read in feature matrix (train & test)

 Train sequences and label feature matrix

In [3]:
#file containing the feat matrix of sequences for training
#tomato
input_file = 'feat_mats/simulated_mg/k40_noninoc_sim5_092622_kmer_feat.csv'

In [4]:
#load csv file in chunks for memory 
%time chunk = pd.read_csv(input_file, header=None, chunksize=1000000)

CPU times: user 2.39 ms, sys: 1.33 ms, total: 3.71 ms
Wall time: 14.1 ms


In [5]:
#concatenate chunks of csv file
%time dataset = pd.concat(chunk)

CPU times: user 51.5 s, sys: 951 ms, total: 52.5 s
Wall time: 52.2 s


In [6]:
#name_column = ['label'] ##this is only for when the label and seq feat mat is together
#rename columns to number of kmer
name_column = []
for i in range(0, len(dataset.columns)):
    name_column.append("kmer" + str(i))
dataset.columns = name_column

In [7]:
dataset.tail()

,kmer0,kmer1,kmer2,kmer3,kmer4,kmer5,kmer6,kmer7,kmer8,kmer9,...,kmer16374,kmer16375,kmer16376,kmer16377,kmer16378,kmer16379,kmer16380,kmer16381,kmer16382,kmer16383
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,2,0,2,0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,2,0,5
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#load file with feat matrix of labels
label_file = 'feat_mats/simulated_mg/k40_noninoc_sim5_092622_label_feat.csv'
#read csv file
label_df = pd.read_csv(label_file, header=None)

In [9]:
#rename column to 'label'
label = label_df.rename({0: 'label'}, axis=1)
label

,label
0,0
1,0
2,1
3,1
4,0
...,...
9995,0
9996,0
9997,0
9998,1


In [10]:
X_train = dataset
Y_train = label

# RandomForest

In [11]:
%time scores = cross_val_score(RandomForestClassifier(), X_train, Y_train.values.ravel(), scoring='accuracy', cv=10)

CPU times: user 4min 9s, sys: 774 ms, total: 4min 9s
Wall time: 4min 9s


In [12]:
score_df = pd.DataFrame(scores)

In [13]:
score_df

,0
0,0.883
1,0.877
2,0.885
3,0.871
4,0.890
5,0.869
6,0.891
7,0.866
8,0.886
9,0.892


In [14]:
#save scores to file
score_df.to_csv('k40_noninoc_sim5_RF_acc_092622.csv', index=False, header=False)